Importing Libraries

In [ ]:
import json
import numpy as np
from collections import defaultdict
from tqdm import tqdm

Parsing file and mounting drive

In [ ]:
def parseData(file):
    for l in open(file):
        yield eval(l)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
location =  "drive/MyDrive/yelp_dataset/"

In [ ]:
f = location +'yelp_academic_dataset_review.json'
#data = list(parseData(f))

In [ ]:
data_uir = [] #data of user item rating rating only
for l in tqdm(parseData(f)):
    data_uir.append((l['user_id'],l['review_id'],l['stars']))
print(len(data_uir))
data_uir[0]

8635403it [07:37, 18861.31it/s]

8635403


('ak0TdVmGKo4pwqdJSTLwWw', 'lWC-xP3rd6obsecCYsGZRg', 4.0)

Shuffling data and creating train and test sets

In [ ]:
import random
random.shuffle(data_uir)

In [ ]:
N = len(data_uir)
train_data = data_uir[0:int(0.8*N)]
test_data = data_uir[int(0.8*N):]

Calculating user and item average on the train set

In [ ]:
user_ratings = defaultdict(list)
item_ratings = defaultdict(list)
global_avg = 0

for d in train_data:
    user,item,rating = d
    user_ratings[user].append(rating)
    item_ratings[item].append(rating)
    global_avg += rating
    
global_avg = global_avg/len(data_uir)

user_avg = defaultdict(float)
item_avg = defaultdict(float)

for u in user_ratings.keys():
    user_avg[u] = np.sum(user_ratings[u])/len(user_ratings[u])

for i in item_ratings.keys():
    item_avg[i] = np.sum(item_ratings[i])/len(item_ratings[i])

Prediction using only user average

In [ ]:
mse = 0
for d in test_data:
    prediction = 0
    user,item,rating = d
    if user in user_avg.keys():
        prediction = user_avg[user]
    else:
        prediction = global_avg
    mse += (prediction - rating)**2
mse = mse/len(test_data)
mse

2.2438710138441995

Prediction using only item average

In [ ]:
mse = 0
for d in test_data:
    prediction = 0
    user,item,rating = d
    if item in item_avg.keys():
        prediction = item_avg[item]
    else:
        prediction = global_avg
    mse += (prediction - rating)**2
mse = mse/len(test_data)
mse

2.680697671811111